# Sprint 論文読解入門

条件  
- 答える際は論文のどの部分からそれが分かるかを書く。
- 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
- 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

<メモ>

Faster R-CNNと呼ばれるオブジェクト検出システムは、2つのモジュールで構成されています。
最初のモジュールは、領域を提案する完全な畳み込みネットワークであり、
2番目のモジュールは、提案された領域を使用する高速R-CNN検出器です[ 2 ]。
システム全体は、オブジェクト検出用の単一の統合ネットワークです（図  2）

実はこれはRegion Proposalの時間を除いた場合の比較でFast R-CNNはRegion Proposalの実行時間が支配的になってしまっている。
一枚の画像に2.3秒かかるが、そのうち2秒(86%!)がRegionProposalに費やされていた。

というのもFast R-CNNではRegion Proposalに従来技術であるSelective Searchを使用しており、そこが速度ボトルネックとなっていた。（コメント有難うございます）
Faster R-CNNはRegionProposalもCNN化することで物体検出モデルを全てDNN化し、高速化するのがモチベーションとなっている。

またFaster-RCNNはMulti-task lossという学習技術を使っており、RegionProposalモデルも込でモデル全体をend-to-endで学習させることに成功している。

## (1) 物体検出の分野にはどういった手法が存在したか。

#### <論文引用>

Recent advances in object detection are driven by the success of region proposal methods (e.g., [4]) and region-based convolutional neural networks (R-CNNs) [5]. Although region-based CNNs were computationally expensive as originally developed in [5], their cost has been drastically reduced thanks to sharing convolutions across proposals [1, 2]. The latest incarnation, Fast R-CNN [2], achieves near real-time rates using very deep networks [3], when ignoring the time spent on region proposals. 

物体検出における最近の進歩は、領域の提案方法の成功によって駆動される（例えば、[ 4 ]）及び領域ベースの畳み込みニューラルネットワーク（R-CNNs）[ 5 ]。元々に開発された領域ベースCNNsは計算上高価であったが[ 5 ]の提案を横切って共有畳み込みに、それらのコストが大幅に低減されているおかげで、[ 1、2 ]。最新の化身であるFast R-CNN [ 2 ]は、非常に深いネットワーク[ 3 ]を使用してほぼリアルタイムのレートを実現します。

#### 答え

R-CNNs、CNNs、Fast R-CNN、Selective Search

#### <その他の情報>

e.g., Selective Search [4] object detectors, R- CNN [5], and Fast R-CNN [2]).

SPPnet [1] and Fast R-CNN 

セレクティブサーチ
https://staff.fnwi.uva.nl/th.gevers/pub/GeversIJCV2013.pdf

## (2) Fasterとあるが、どういった仕組みで高速化したのか。



#### <論文引用>

Our object detection system, called Faster R-CNN, is composed of two modules. The first module is a deep fully convolutional network that proposes regions, and the second module is the Fast R-CNN detector [2] that uses the proposed regions. The entire system is a single, unified network for object detection (Figure 2). Using the recently popular terminology of neural networks with ‘attention’ [31] mechanisms, the RPN module tells the Fast R-CNN module where to look. In Section 3.1 we introduce the designs and properties of the network for region proposal. In Section 3.2 we develop algorithms for training both modules with features shared.

Faster R-CNNと呼ばれるオブジェクト検出システムは、2つのモジュールで構成されています。最初のモジュールは、領域を提案する完全な畳み込みネットワークであり、2番目のモジュールは、提案された領域を使用する高速R-CNN検出器です[ 2 ]。システム全体は、オブジェクト検出用の単一の統合ネットワークです（図  2）。「注意」[ 31 ]メカニズムを備えたニューラルネットワークの最近人気のある用語を使用して、RPNモジュールはFast R-CNNモジュールにどこを見ればよいかを伝えます。セクション  3.1では、地域提案のためのネットワークの設計と特性を紹介します。セクション3.2では  、機能を共有して両方のモジュールをトレーニングするアルゴリズムを開発します。

物体検出にもNNを利用

#### その他

Region Proposal Network (RPN)という物体候補領域を推定してくれるネットワーク+ RoI Poolingにクラス推定を行うことでEnd to Endで学習できるアーキテクチャを提案しました。

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。



#### 論文引用

One-Stage Detection vs. Two-Stage Proposal + Detection. The OverFeat paper [9] proposes a detection method that uses regressors and classifiers on sliding windows over convolutional feature maps. OverFeat is a one-stage, class-specific detection pipeline, and ours is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. In OverFeat, the region-wise features come from a sliding window of one aspect ratio over a scale pyramid. These features are used to simultaneously determine the location and category of objects. In RPN, the features are from square (3
×
3) sliding windows and predict proposals relative to anchors with different scales and aspect ratios. Though both methods use sliding windows, the region proposal task is only the first stage of Faster R-CNN—the downstream Fast R-CNN detector attends to the proposals to refine them. In the second stage of our cascade, the region-wise features are adaptively pooled [1, 2] from proposal boxes that more faithfully cover the features of the regions. We believe these features lead to more accurate detections.
To compare the one-stage and two-stage systems, we emulate the OverFeat system (and thus also circumvent other differences of implementation details) by one-stage Fast R-CNN. In this system, the “proposals” are dense sliding windows of 3 scales (128, 256, 512) and 3 aspect ratios (1:1, 1:2, 2:1). Fast R-CNN is trained to predict class-specific scores and regress box locations from these sliding windows. Because the OverFeat system adopts an image pyramid, we also evaluate using convolutional features extracted from 5 scales. We use those 5 scales as in [1, 2].
Table X compares the two-stage system and two variants of the one-stage system. Using the ZF model, the one-stage system has an mAP of 53.9%. This is lower than the two-stage system (58.7%) by 4.8%. This experiment justifies the effectiveness of cascaded region proposals and object detection. Similar observations are reported in [2, 39], where replacing SS region proposals with sliding windows leads to 
∼
6% degradation in both papers. We also note that the one-stage system is slower as it has considerably more proposals to process.

一段階検出VS。2段階の提案+検出。 OverFeat論文[ 9 ]は、畳み込み特徴マップ上のスライディングウィンドウでリグレッサとクラシファイアを使用する検出方法を提案しています。OverFeatは、一段階、クラス固有の検出パイプライン、そして我々は、二段カスケードクラスにとらわれない提案とクラス固有の検出からなります。OverFeatでは、リージョン単位の機能は、スケールピラミッド上の1つのアスペクト比のスライドウィンドウから取得されます。これらの機能は、オブジェクトの場所とカテゴリを同時に決定するために使用されます。RPNでは、フィーチャは正方形（3 
×
3）ウィンドウをスライドさせ、異なるスケールとアスペクト比のアンカーに関連する提案を予測します。両方の方法は、スライディングウィンドウを使用しても、領域提案タスクは、R-CNN-下流高速R-CNN検出高速の最初の段階である出席それらをさらに絞り込むことが提案されます。我々のカスケードの第2段階では、領域ごとの特徴が適応的にプールされる[ 1、2 ]より忠実領域の特徴をカバー提案ボックスから。これらの機能は、より正確な検出につながると考えています。

1段階システムと2段階システムを比較するために、1 段階高速R-CNN によってOverFeatシステムをエミュレートします（したがって、実装の詳細の他の違いを回避します）。このシステムでは、「提案」は3つのスケール（128、256、512）および3つのアスペクト比（1：1、1：2、2：1）の密なスライドウィンドウです。Fast R-CNNは、これらのスライディングウィンドウからクラス固有のスコアと回帰ボックスの位置を予測するようにトレーニングされています。OverFeatシステムは画像ピラミッドを採用しているため、5つのスケールから抽出された畳み込み機能を使用して評価します。我々は、同様にそれらの5つのスケールを使用し、[ 1、2 ]。

表  Xは、2段階システムと1段階システムの2つのバリアントを比較しています。ZFモデルを使用すると、1ステージシステムのmAPは53.9％になります。これは、2段階システム（58.7％）よりも4.8％低いです。この実験は、カスケード領域の提案とオブジェクト検出の有効性を正当化します。同様の観察がで報告されている[ 2、39 ]に窓リードを摺動してSS領域の提案を交換する場合、
〜
の両方紙6％の分解。また、処理する提案がかなり多いため、1段階システムの方が遅いことにも注意してください。

#### その他

two stage detection framework: 最初に領域候補の推定(region proposal)を行い、次にクラスやbounding boxの座標を推定する。
one stage detection framework (region proposal free framework): 領域候補の推定と、クラスやbounding boxの座標の推定を1段階で行う。

## (4) RPNとは何か。



#### 論文引用

A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.3 We model this process with a fully convolutional network [7], which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. In our experiments, we investigate the Zeiler and Fergus model [32] (ZF), which has 5 shareable convolutional layers and the Simonyan and Zisserman model [3] (VGG-16), which has 13 shareable convolutional layers.
To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an 
n
×
n
 spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature (256-d for ZF and 512-d for VGG, with ReLU [33] following). This feature is fed into two sibling fully-connected layers—a box-regression layer (reg) and a box-classification layer (cls). We use 
n
=
3
 in this paper, noting that the effective receptive field on the input image is large (171 and 228 pixels for ZF and VGG, respectively). This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an 
n
×
n
convolutional layer followed by two sibling 
1
×
1
 convolutional layers (for reg and cls, respectively).

領域提案ネットワーク（RPN）は、入力として（任意のサイズの）画像を受け取り、それぞれがオブジェクト性スコアを持つ一連の長方形のオブジェクト提案を出力します。3このセクションで説明 する完全な畳み込みネットワーク[ 7 ]を使用して、このプロセスをモデル化します。最終的な目標は計算を高速R-CNNオブジェクト検出ネットワーク[ 2 ]と共有することであるため、両方のネットが畳み込み層の共通セットを共有すると仮定します。実験では、5つの共有可能な畳み込み層を持つZeiler and Fergusモデル[ 32 ]（ZF）と、Simonyan and Zissermanモデル[ 3 ]を調査します。 （VGG-16）、13の共有可能な畳み込み層があります。

地域の提案を生成するために、最後の共有畳み込み層によって出力された畳み込み特徴マップ上に小さなネットワークをスライドさせます。この小さなネットワークは、入力として、入力畳み込み特徴マップの
n 
× 
n
空間ウィンドウを取ります。各スライディングウィンドウは、低次元のフィーチャにマッピングされます（ZFの場合は256-d、VGGの場合は512-d、ReLU [ 33 ]が後に続きます）。この機能は、完全に接続された2つの兄弟層（ボックス回帰層（reg）とボックス分類層（cls））に供給されます。
n 
= 
3
を使用します
このホワイトペーパーでは、入力画像の有効な受容野が大きいことに注意してください（ZFおよびVGGでそれぞれ171および228ピクセル）。このミニネットワークは、図3（左）の単一の位置に示されてい  ます。ミニネットワークはスライディングウィンドウ方式で動作するため、完全に接続されたレイヤーはすべての空間ロケーションで共有されることに注意してください。このアーキテクチャは、
n 
× 
n
畳み込み層とそれに続く2つの兄弟の
1 
× 
1
畳み込み層（それぞれregとclsの場合）で自然に実装されます。

#### その他

本文から
入力として（任意のサイズの）画像を受け取り、
それぞれがオブジェクト性スコアを持つ一連の長方形のオブジェクト提案を出力します  
RPNは、物体候補を出力するために(1)物体かどうかを表すスコア(図中cls layer)と
(2)物体の領域(図中reg layer)の２つを同時に出力するように設計されています。
画像全体のfeature mapsから予め決められたk個の固定枠(Anchor)を用いて特徴を抽出し、
RPNの入力とすることで、各場所において物体候補とすべきかどうかを推定します。

物体候補として推定された出力枠(reg layer)の範囲を、
Fast R-CNN同様にRoI Poolingしクラス識別用のネットワークの入力とすることで最終的な物体検出を実現します。
物体候補検出がDeep化されたことで、既存手法(Selective Search)[7]よりも物体候補が高精度化&候補数が少なくなり、 
GPU上で5fpsの実行速度(VGGのネットワークを利用)を達成しました。 また、識別精度もFast-RCNNより高精度化しています。



Region Propsal Network(RPN)はCNNの特徴マップ（つまりResnet等のCNN層出力）を入力。

ある領域が物体か背景か(objectness)およびアンカーの位置の補正データ(corrdinates)を出力する。objectnessは0-1の値で1に近いほど物体である確証が高い。corrdinatesはBindingBoxの四角の座標について補正する量を出力する。

## (5) RoIプーリングとは何か。



#### 論文引用

Nevertheless, our method achieves bounding-box
regression by a different manner from previous RoIbased (Region of Interest) methods [1], [2]. In [1],
[2], bounding-box regression is performed on features
pooled from arbitrarily sized RoIs, and the regression
weights are shared by all region sizes. In our formulation, the features used for regression are of the same
spatial size (3 × 3) on the feature maps. To account
for varying sizes, a set of k bounding-box regressors
are learned. Each regressor is responsible for one scale
and one aspect ratio, and the k regressors do not share
weights. As such, it is still possible to predict boxes of
various sizes even though the features are of a fixed
size/scale, thanks to the design of anchors.  
As discussed
above, the bounding boxes predicted by RPN are
also functions of the input. The RoI pooling layer
[2] in Fast R-CNN accepts the convolutional features
and also the predicted bounding boxes as input, so
a theoretically valid backpropagation solver should
also involve gradients w.r.t. the box coordinates. These
gradients are ignored in the above approximate joint
training. In a non-approximate joint training solution,
we need an RoI pooling layer that is differentiable
w.r.t. the box coordinates. This is a nontrivial problem
and a solution can be given by an “RoI warping” layer
as developed in [15], which is beyond the scope of this
paper.

私たちの方法は境界ボックスを達成します
以前のRoIbased（関心のある地域）メソッド[1]、[2]とは異なる方法による回帰。 1で]、
[2]、境界ボックス回帰はフィーチャに対して実行されます
任意のサイズのRoIからプールされ、回帰
重みはすべての領域サイズで共有されます。私たちの定式化では、回帰に使用される特徴は同じです
機能マップ上の空間サイズ（3×3）。アカウントへ
さまざまなサイズの場合、k個の境界ボックスリグレッサのセット
学びます。各リグレッサーは1つのスケールを担当します
そして1つのアスペクト比、およびk個のリグレッサは共有しません
重み。そのため、次のボックスを予測することは依然として可能です。
機能が固定されていても、さまざまなサイズ
アンカーの設計のおかげでサイズ/スケール。

#### その他

RoI Poolingでは、ある程度畳み込み処理を行ったfeature mapから、region proposalにあたる部分領域をうまく
「固定サイズのfeature map」として抽出します。
https://qiita.com/yu4u/items/5cbe9db166a5d72f9eb8




## (6) Anchorのサイズはどうするのが適切か。



#### 論文引用

The anchor boxes that cross image boundaries need
to be handled with care. During training, we ignore
all cross-boundary anchors so they do not contribute
to the loss. For a typical 1000 × 600 image, there
will be roughly 20000 (≈ 60 × 40 × 9) anchors in
total. With the cross-boundary anchors ignored, there
are about 6000 anchors per image for training. If the
boundary-crossing outliers are not ignored in training,
they introduce large, difficult to correct error terms in
the objective, and training does not converge. During
testing, however, we still apply the fully convolutional
RPN to the entire image. This may generate crossboundary proposal boxes, which we clip to the image
boundary.

画像の境界を越えるアンカーボックスが必要
注意して取り扱ってください。トレーニング中、無視します
すべての境界を越えたアンカーなので、貢献しません
損失に。典型的な1000×600画像の場合、
約20000（≈60×40×9）アンカーになります
合計。境界を越えたアンカーを無視して、そこに
トレーニング用の画像ごとに約6000個のアンカーがあります。もし
境界を越える外れ値はトレーニングでは無視されません。
修正が困難な大きなエラー用語を導入します
目的、およびトレーニングは収束しません。中に
ただし、テストでは、完全な畳み込みを適用します
画像全体に対するRPN。これにより、境界を越えた提案ボックスが生成され、画像にクリップされます。
境界。

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。



#### 論文引用

We present more results on the Microsoft COCO
object detection dataset [12]. This dataset involves 80
object categories. We experiment with the 80k images
on the training set, 40k images on the validation set,
and 20k images on the test-dev set. We evaluate the
mAP averaged for IoU ∈ [0.5 : 0.05 : 0.95] (COCO’s
standard metric, simply denoted as mAP@[.5, .95])
and mAP@0.5 (PASCAL VOC’s metric).

Microsoft COCOオブジェクト検出データセット[ 12 ]でより多くの結果を提示します。このデータセットには80のオブジェクトカテゴリが含まれます。トレーニングセットの80k画像、検証セットの40k画像、test-devセットの20k画像で実験します。私たちは、マップがIOUのための平均評価
∈ 
[ 
0.5 
：
0.05 
：
0.95 
]
（COCOの標準メトリック、単にと表記MAP @ [0.5、0.95]）とmAP@0.5 （PASCAL VOCの測定基準）。


In Table 11 we first report the results of the Fast
R-CNN system [2] using the implementation in this
paper. Our Fast R-CNN baseline has 39.3% mAP@0.5
on the test-dev set, higher than that reported in [2].
We conjecture that the reason for this gap is mainly
due to the definition of the negative samples and also
the changes of the mini-batch sizes. We also note that
the mAP@[.5, .95] is just comparable.
Next we evaluate our Faster R-CNN system. Using
the COCO training set to train, Faster R-CNN has
42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the
COCO test-dev set. This is 2.8% higher for mAP@0.5
and 2.2% higher for mAP@[.5, .95] than the Fast RCNN counterpart under the same protocol (Table 11).
This indicates that RPN performs excellent for improving the localization accuracy at higher IoU thresholds. Using the COCO trainval set to train, Faster RCNN has 42.7% mAP@0.5 and 21.9% mAP@[.5, .95] on
the COCO test-dev set. Figure 6 shows some results
on the MS COCO test-dev set.

表  XIでは、最初に、このペーパーの実装を使用したFast R-CNNシステム[ 2 ]の結果を報告します。Fast R-CNNベースラインのtest-devセットのmAP@0.5 は39.3％で、[ 2 ]で報告されたものよりも高くなっています。このギャップの原因は、主にネガティブサンプルの定義とミニバッチサイズの変更によるものと推測します。また、mAP @ [.5、.95]は単なる比較にすぎないことに注意してください。

次に、より高速なR-CNNシステムを評価します。COCOトレーニングセットを使用してトレーニングを行うと、Faster R-CNNはCOCOテスト開発セットで42.1％mAP@0.5 および21.5％mAP @ [.5、.95]になります。これは2.8％高いmAP@0.5 および2.2％より高いのためのマップ@ [0.5、0.95]同じプロトコル（表の下の高速R-CNN対応物より  XI）。これは、RPNが高いIoUしきい値でのローカライズの精度を向上させるために優れたパフォーマンスを発揮することを示しています。COCO trainvalセットを使用してトレーニングを行うと、Faster R-CNNはCOCO test-devセットで42.7％mAP@0.5 および21.9％mAP @ [.5、.95]になります。図  6は、MS COCO test-devセットのいくつかの結果を示しています。